In [180]:
import warnings
warnings.simplefilter("ignore")

import datetime, os
from functools import partial
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow import keras
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [2]:
# load the TensorBoard notebook extension
%load_ext tensorboard

In [181]:
# Set paths

DATA_PATH = '/Users/alex791/Library/CloudStorage/OneDrive-PNNL/Documents/Projects/ML_Earth_Projects/Reservoir_Project/Data'
HP_TUNING_PATH = '/Users/alex791/Library/CloudStorage/OneDrive-PNNL/Documents/Projects/ML_Earth_Projects/Reservoir_Project/Hyperparameter_Tuning'

In [182]:
basin_inflow_train = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_train.xlsx', index_col=0)
basin_inflow_validation = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_validation.xlsx', index_col=0)
basin_inflow_test = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_test.xlsx', index_col=0)

In [183]:
basin_inflow_train.head()

,INFLOW,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,ADR_TEMP_MIN,HYS_PRECIP_ACC,HYS_PRECIP_INCR,HYS_SNOW_DEPTH,HYS_SNOW_WATER_CONTENT,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
0,0.044963,-0.289112,0.340301,0.054044,-0.375165,0.305438,-0.011090,0.388324,-0.123764,0.061586,...,0.108819,0.183304,-0.491096,-0.610589,-0.458829,-0.078244,0.224818,-0.039676,-0.564907,0.045612
1,0.045396,-0.283597,0.376414,-0.093752,-0.373458,0.056615,-0.006376,0.419682,-0.106719,0.069362,...,0.113669,0.185770,-0.363760,-0.448188,-0.336938,-0.073231,0.299318,-0.030774,-0.682164,0.086839
2,0.039165,-0.281148,0.243520,-0.113826,-0.227147,-0.029060,-0.001178,0.442274,-0.093225,0.079873,...,0.118205,0.188094,-0.202812,-0.252427,-0.180427,-0.068255,0.297508,-0.093725,-0.530415,0.041245
3,0.022834,-0.290972,-0.313413,-0.146230,-0.155065,-0.157082,-0.009333,-0.549211,-0.092705,0.087465,...,0.116945,0.188011,-0.136433,-0.141606,-0.138177,-0.078026,-0.428531,-0.105323,-0.350378,-0.056045
4,-0.014312,-0.300133,-0.389419,-0.063319,-0.042481,-0.134920,-0.015781,-0.591777,-0.109237,0.075137,...,0.127933,0.188631,-0.087231,-0.053518,-0.108681,-0.086058,-0.465569,0.009074,-0.102076,-0.086963


### Data windowing

In [184]:
# TensorFlow utility class for producing data windows from time series data

class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}
    
    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

In [185]:
"""
Window
- Given 60 days of history predict 30 days into the future. Why? A season is about 90 days in a CA WY
- Window size: 90
"""

window = WindowGenerator(input_width=60, label_width=1, shift=30,
                     train_df=basin_inflow_train, val_df=basin_inflow_validation, 
                     test_df=basin_inflow_test, label_columns=['INFLOW'])

window

In [186]:
# create a window of inputs and labels

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # set shapes after slicing
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [187]:
# create a dataset of sliding windows over a time series dataframe

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  # (input_window, label_window) pairs 
  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [188]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test

In [189]:
# each element is an (inputs, label) pair
window.train.element_spec

(TensorSpec(shape=(None, 60, 36), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))

In [190]:
# example batch
for inputs, labels in window.train.take(1):
  print(f'Inputs shape (batch, time, features): {inputs.shape}')
  print(f'Labels shape (batch, time, features): {labels.shape}')

Inputs shape (batch, time, features): (32, 60, 36)
Labels shape (batch, time, features): (32, 1, 1)


### Modeling

In [191]:
val_performance = {}
test_performance = {}

#### Create baseline

In [192]:
# tensorflow baseline utility class for data windowing

class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [193]:
baseline = Baseline(label_index=window.column_indices['INFLOW'])

baseline.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance['Baseline'] = baseline.evaluate(window.val)
test_performance['Baseline'] = baseline.evaluate(window.test, verbose=0)

21/21 [==============================] - 0s 488us/step - loss: 0.1517 - mean_absolute_error: 0.1517


#### LSTM

In [194]:
MAX_EPOCHS = 10

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                optimizer=tf.keras.optimizers.legacy.Adam())

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping, tensorboard_callback])
  return history

LSTM 1

In [195]:
"""
Inputs shape (batch, time, features): (32, 60, 36) - 32 batch size, 60 time steps, 36 features
"""

lstm_v1 = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 36]),
    tf.keras.layers.Dense(units=1)
])

In [196]:
history_lstm_v1 = compile_and_fit(lstm_v1, window)

Epoch 1/10
79/79 [==============================] - 1s 9ms/step - loss: 0.0628 - val_loss: 0.1552
Epoch 2/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0452 - val_loss: 0.1385
Epoch 3/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0414 - val_loss: 0.1315
Epoch 4/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0399 - val_loss: 0.1284
Epoch 5/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0374 - val_loss: 0.1295
Epoch 6/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0362 - val_loss: 0.1387


In [ ]:
%tensorboard --logdir logs

In [198]:
val_performance['LSTM1'] = lstm_v1.evaluate(window.val)
test_performance['LSTM1'] = lstm_v1.evaluate(window.test, verbose=0)

print('LSTM 1 Validation performance: ', val_performance['LSTM1'])
print('LSTM 1 Test performance: ', test_performance['LSTM1'])

21/21 [==============================] - 0s 2ms/step - loss: 0.1387
LSTM 1 Validation performance:  0.13874830305576324
LSTM 1 Test performance:  0.14985796809196472


LSTM 2

In [199]:
lstm_v2 = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 36]),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(units=1)
])

In [200]:
history_lstm_v2 = compile_and_fit(lstm_v2, window)

Epoch 1/10
79/79 [==============================] - 2s 17ms/step - loss: 0.0430 - val_loss: 0.1555
Epoch 2/10
79/79 [==============================] - 1s 14ms/step - loss: 0.0321 - val_loss: 0.1474
Epoch 3/10
79/79 [==============================] - 1s 14ms/step - loss: 0.0297 - val_loss: 0.1514
Epoch 4/10
79/79 [==============================] - 1s 14ms/step - loss: 0.0257 - val_loss: 0.1452
Epoch 5/10
79/79 [==============================] - 1s 14ms/step - loss: 0.0229 - val_loss: 0.1514
Epoch 6/10
79/79 [==============================] - 1s 14ms/step - loss: 0.0203 - val_loss: 0.1527


In [29]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 45646), started 0:23:25 ago. (Use '!kill 45646' to kill it.)

In [201]:
val_performance['LSTM2'] = lstm_v2.evaluate(window.val)
test_performance['LSTM2'] = lstm_v2.evaluate(window.test, verbose=0)

print('LSTM 2 Validation performance: ', val_performance['LSTM2'])
print('LSTM 2 Test performance: ', test_performance['LSTM2'])

21/21 [==============================] - 0s 3ms/step - loss: 0.1527
LSTM 2 Validation performance:  0.15269407629966736
LSTM 2 Test performance:  0.09494537860155106


LSTM 3

In [202]:
RegularizedLSTM = partial(tf.keras.layers.LSTM,
                           dropout=0.01,
                           recurrent_dropout=0.01,
                           kernel_regularizer=tf.keras.regularizers.l2(0.05))

In [203]:
lstm_v3 = tf.keras.models.Sequential([
    RegularizedLSTM(32, return_sequences=True, input_shape=[None, 36]),
    RegularizedLSTM(32),
    tf.keras.layers.Dense(units=1)
])

In [204]:
history_lstm_v3 = compile_and_fit(lstm_v3, window)

Epoch 1/10
79/79 [==============================] - 4s 34ms/step - loss: 3.0731 - val_loss: 1.5350
Epoch 2/10
79/79 [==============================] - 3s 34ms/step - loss: 0.7713 - val_loss: 0.4343
Epoch 3/10
79/79 [==============================] - 3s 34ms/step - loss: 0.1850 - val_loss: 0.1943
Epoch 4/10
79/79 [==============================] - 3s 34ms/step - loss: 0.0637 - val_loss: 0.1485
Epoch 5/10
79/79 [==============================] - 3s 35ms/step - loss: 0.0437 - val_loss: 0.1551
Epoch 6/10
79/79 [==============================] - 3s 34ms/step - loss: 0.0394 - val_loss: 0.1457
Epoch 7/10
79/79 [==============================] - 3s 34ms/step - loss: 0.0378 - val_loss: 0.1461
Epoch 8/10
79/79 [==============================] - 3s 34ms/step - loss: 0.0378 - val_loss: 0.1465


In [89]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 45646), started 2:07:43 ago. (Use '!kill 45646' to kill it.)

In [205]:
val_performance['LSTM3'] = lstm_v3.evaluate(window.val)
test_performance['LSTM3'] = lstm_v3.evaluate(window.test, verbose=0)

print('LSTM 3 Validation performance: ', val_performance['LSTM3'])
print('LSTM 3 Test performance: ', test_performance['LSTM3'])

21/21 [==============================] - 0s 4ms/step - loss: 0.1465
LSTM 3 Validation performance:  0.14650316536426544
LSTM 3 Test performance:  0.07470374554395676


#### Hyperparameter tuning

In [206]:
def build_model(hp):
  lstm = keras.Sequential()

  ### Tuning hyperparameters ### 

  # Number of lstm layer units
  hp_units = hp.Int('units', min_value=32, max_value=128, step=32)

  # Dropout rate applied to input values
  hp_dropout = hp.Choice("dropout", [0.2, 0.3, 0.4, 0.5])

  # Recurrent dropout rate applied to hidden cell states between time steps
  hp_recurrent_dropout = hp.Choice("recurrent_dropout", [0.2, 0.3, 0.4, 0.5])
  
  # L2 regularization 
  hp_l2_reg = hp.Choice("l2", [0.001, 0.01, 0.02, 0.05])

  # Optimizer learning rate
  hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

  ### Layers ### 
  
  lstm.add(tf.keras.layers.LSTM(units=hp_units, dropout=hp_dropout, recurrent_dropout=hp_recurrent_dropout, 
                  kernel_regularizer=tf.keras.regularizers.l2(l2=hp_l2_reg),
                  return_sequences=True, input_shape=[None, 36]))
    
  lstm.add(tf.keras.layers.LSTM(units=hp_units, dropout=hp_dropout, recurrent_dropout=hp_recurrent_dropout, 
                  kernel_regularizer=tf.keras.regularizers.l2(l2=hp_l2_reg)))
    
  lstm.add(keras.layers.Dense(1))    
    
  ### Compile ### 
           
  lstm.compile(loss=tf.keras.losses.MeanAbsoluteError(),
               optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=hp_learning_rate),
               metrics=['mean_absolute_error'])

  return lstm

In [208]:
tuner = kt.Hyperband(build_model,
                     objective='val_mean_absolute_error',
                     max_epochs=10,
                     factor=3,
                     directory=HP_TUNING_PATH,
                     project_name='reservoir_model_hp_tuning_v2')

In [209]:
MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [210]:
# args for search are those used for model fit
tuner.search(window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping])

Trial 30 Complete [00h 01m 50s]
val_mean_absolute_error: 0.11860812455415726

Best val_mean_absolute_error So Far: 0.11121592670679092
Total elapsed time: 00h 14m 04s


In [211]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete and below are the optimal values:

- Units: {best_hps.get('units')}
- Dropout: {best_hps.get('dropout')}
- Recurrent dropout: {best_hps.get('recurrent_dropout')}
- L2: {best_hps.get('l2')}
- Learning rate: {best_hps.get('learning_rate')}
""")


The hyperparameter search is complete and below are the optimal values:

- Units: 32
- Dropout: 0.2
- Recurrent dropout: 0.5
- L2: 0.01
- Learning rate: 0.001



#### Train

In [212]:
# first find the optimal number of training epochs

MAX_EPOCHS = 10

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=2,
                                                mode='min')

model = tuner.hypermodel.build(best_hps)
history = model.fit(window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping])

val_mae_per_epoch = history.history['val_mean_absolute_error']
best_epoch = val_mae_per_epoch.index(min(val_mae_per_epoch)) + 1 # find epoch of lowest validation MAE 
print('Best epoch: %d' % (best_epoch,))

Epoch 1/10
79/79 [==============================] - 4s 34ms/step - loss: 0.7020 - mean_absolute_error: 0.0479 - val_loss: 0.4679 - val_mean_absolute_error: 0.1227
Epoch 2/10
79/79 [==============================] - 3s 34ms/step - loss: 0.2459 - mean_absolute_error: 0.0383 - val_loss: 0.2452 - val_mean_absolute_error: 0.1337
Epoch 3/10
79/79 [==============================] - 3s 35ms/step - loss: 0.1085 - mean_absolute_error: 0.0374 - val_loss: 0.1761 - val_mean_absolute_error: 0.1333
Epoch 4/10
79/79 [==============================] - 3s 35ms/step - loss: 0.0673 - mean_absolute_error: 0.0368 - val_loss: 0.1562 - val_mean_absolute_error: 0.1351
Epoch 5/10
79/79 [==============================] - 3s 35ms/step - loss: 0.0526 - mean_absolute_error: 0.0362 - val_loss: 0.1471 - val_mean_absolute_error: 0.1347
Epoch 6/10
79/79 [==============================] - 3s 35ms/step - loss: 0.0455 - mean_absolute_error: 0.0352 - val_loss: 0.1465 - val_mean_absolute_error: 0.1381
Epoch 7/10
79/79 [====

In [213]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=2,
                                                mode='min')
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

hypermodel = tuner.hypermodel.build(best_hps)
history = hypermodel.fit(window.train, epochs=best_epoch, validation_data=window.val, 
                         callbacks=[early_stopping, tensorboard_callback])

79/79 [==============================] - 4s 35ms/step - loss: 0.7018 - mean_absolute_error: 0.0480 - val_loss: 0.4654 - val_mean_absolute_error: 0.1236


In [214]:
val_performance['Hypermodel'] = hypermodel.evaluate(window.val)
test_performance['Hypermodel'] = hypermodel.evaluate(window.test, verbose=0)

print('Hypermodel Validation performance: ', val_performance['Hypermodel'])
print('Hypermodel Test performance: ', test_performance['Hypermodel'])

21/21 [==============================] - 0s 4ms/step - loss: 0.4654 - mean_absolute_error: 0.1236
Hypermodel Validation performance:  [0.4653622806072235, 0.1235504150390625]
Hypermodel Test performance:  [0.38941654562950134, 0.04760463908314705]


### Inference

In [215]:
# inverse normalization

def inverse_scaling(data):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    fitted_scaler = scaler.fit(data)
    inversed_data = fitted_scaler.inverse_transform(data)
    return inversed_data

In [216]:
window.test.take(1) # given 60 days of info, make prediction. Batch of 32.

<_TakeDataset element_spec=(TensorSpec(shape=(None, 60, 36), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))>

In [217]:
# make predictions

testData = window.test.take(1)
testPredict = hypermodel.predict(testData)

1/1 [==============================] - 0s 158ms/step


In [218]:
testPredict

array([[-0.03850911],
       [-0.08616414],
       [-0.03508677],
       [ 0.05452147],
       [ 0.04328771],
       [ 0.00158038],
       [-0.07983816],
       [-0.08319991],
       [ 0.01772927],
       [-0.08097013],
       [ 0.04561995],
       [-0.0857792 ],
       [-0.13102858],
       [ 0.02652507],
       [-0.06717805],
       [ 0.02680867],
       [-0.06366231],
       [-0.01168345],
       [-0.02678484],
       [ 0.02099827],
       [-0.0386314 ],
       [ 0.02127475],
       [-0.07938855],
       [-0.03992091],
       [ 0.02334642],
       [-0.05533672],
       [-0.07522446],
       [ 0.01150507],
       [-0.06335182],
       [ 0.05045534],
       [ 0.01252339],
       [-0.0454586 ]], dtype=float32)

In [219]:
### Predicted Inflow ###

# predictedInflow = testPredict.flatten() # Batch of 32 output
predictedInflow = inverse_scaling(testPredict).flatten().tolist() # Batch of 32 output
predictedInflow

[-0.04182623699307442,
 -0.04624743387103081,
 -0.04150872677564621,
 -0.03319532424211502,
 -0.034237537533044815,
 -0.03810693323612213,
 -0.04566054046154022,
 -0.04597242921590805,
 -0.03660871833562851,
 -0.04576556012034416,
 -0.03402116149663925,
 -0.04621171951293945,
 -0.05040973424911499,
 -0.03579268977046013,
 -0.044485997408628464,
 -0.03576637804508209,
 -0.044159822165966034,
 -0.039337486028671265,
 -0.04073851555585861,
 -0.03630543872714043,
 -0.04183758422732353,
 -0.036279790103435516,
 -0.045618828386068344,
 -0.04195721447467804,
 -0.03608758747577667,
 -0.04338741675019264,
 -0.04523250460624695,
 -0.03718617185950279,
 -0.044131018221378326,
 -0.03357255831360817,
 -0.0370916947722435,
 -0.04247097298502922]

In [220]:
### Actual Inflow ### 

windowTest = window.test.take(1); # Batch of 32
target = [] # predicting 30 days ahead. Batch of 32.

for inputs, labels in windowTest.as_numpy_iterator():
  target = [np.asarray(labels).flatten()]
  print('Before scaling: ', target)

actualInflow = inverse_scaling(target).flatten().tolist()
print('\nActual inflow: ', actualInflow)

Before scaling:  [array([-0.01912978,  0.01490423,  0.01460248,  0.03595458,  0.02328497,
       -0.03500219, -0.05108545, -0.00266521, -0.05396525, -0.10252491,
        0.011954  , -0.12387791, -0.03209262, -0.00138498, -0.19309357,
       -0.10602447,  0.0244406 , -0.02554115,  0.03212425, -0.1348074 ,
       -0.20092668,  0.02369722, -0.0714507 , -0.05879815, -0.05191278,
       -0.00870498, -0.09456588,  0.03157554,  0.01557928,  0.00769655,
        0.01478095, -0.0250665 ], dtype=float32)]

Actual inflow:  [0.47130533400923014, 0.5223563378676772, 0.5219037220813334, 0.553931875154376, 0.5349274575710297, 0.44749671407043934, 0.42337181977927685, 0.4960021789884195, 0.41905212216079235, 0.34621262922883034, 0.5179310017265379, 0.3141831308603287, 0.45186106488108635, 0.49792252416955307, 0.21035964787006378, 0.34096330031752586, 0.5366609022021294, 0.4616882763803005, 0.5481863822788, 0.29778891056776047, 0.19860998541116714, 0.5355458296835423, 0.3928239457309246, 0.4118027705699

In [ ]:
# compare predicted and actual inflow values 

mean_absolute_error(actualInflow, predictedInflow)